##### MOVIE RATE PREDICTION WITH PYTHON #####

In [1]:
###Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_absolute_error
import re

####LOAD THE DATASET####


In [2]:
file_path = r'C:\Users\HP\AppData\Local\Microsoft\Windows\INetCache\IE\TE0HON1L\IMDb_Movies_India[1].csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Display the first few rows to verify successful loading
df.head()


,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali



# Step 1: Data Cleaning and Preprocessing

In [3]:
#Remove rows with missing target 'Rating'
movies_df = df.dropna(subset=['Rating'])

#Convert 'Year' to a numeric column (extract numeric part if necessary)
movies_df['Year'] = pd.to_numeric(movies_df['Year'].str.extract('(\d+)', expand=False), errors='coerce')

#Convert 'Duration' to numeric (remove 'min' if present and handle errors)
movies_df['Duration'] = pd.to_numeric(movies_df['Duration'].str.replace(' min', ''), errors='coerce')

#Convert 'Votes' to numeric
movies_df['Votes'] = pd.to_numeric(movies_df['Votes'], errors='coerce')

#Drop rows where essential features are missing (optional)
movies_df = movies_df.dropna(subset=['Duration', 'Year', 'Votes', 'Genre', 'Director'])

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HP\AppData\Local\Temp\ipykernel_8940\267173085.py:5: SyntaxWarning: invalid escape sequence '\d'
  movies_df['Year'] = pd.to_numeric(movies_df['Year'].str.extract('(\d+)', expand=False), errors='coerce')
C:\Users\HP\AppData\Local\Temp\ipykernel_8940\267173085.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['Year'] = pd.to_numeric(movies_df['Year'].str.extract('(\d+)', expand=False), errors='coerce')
C:\Users\HP\AppData\Local\Temp\ipykernel_8940\267173085.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

# Step 2: Feature Engineering

In [4]:
# 2.1 Encoding categorical features (e.g., Director, Genre)
# Use CountVectorizer for text-based categorical columns like Genre and Director

# Encode Genre using CountVectorizer (Multi-label binary encoding)
genre_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
genre_features = genre_vectorizer.fit_transform(movies_df['Genre'].fillna('')).toarray()
genre_feature_names = genre_vectorizer.get_feature_names_out()

# Convert Genre features to DataFrame
genre_df = pd.DataFrame(genre_features, columns=genre_feature_names)

# Concatenate genre_df with the original dataframe
movies_df = pd.concat([movies_df.reset_index(drop=True), genre_df], axis=1)


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Step 3: Prepare features and target

In [5]:
X = movies_df[['Year', 'Duration', 'Votes'] + list(genre_df.columns)]
y = movies_df['Rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Step 4: Build and Train the Model

In [6]:
# Instantiate a model - you can start with a simple linear model, then try RandomForest for better performance
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

# Step 5: Evaluate the Model

In [7]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate using MAE and RMSE
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Absolute Error (MAE): 0.8373421722846441
Root Mean Squared Error (RMSE): 1.1012961119901603
